# Google Drive

This first code block attaches your google drive and makes a folder structure. You only need to run this when a new VM is assigned to you. To get your code as a single python file go through the following menus File->'Download .py'.

This also downloads the 5 files that contain the dataset and the checkpoint:


https://drive.google.com/open?id=1uBWazGxSZgWs70JjSWBu-KZwy5sAcxLh

https://drive.google.com/open?id=18Bh2qwVwdDwu7JK_plrnpAz9KjA5gWkv

https://drive.google.com/open?id=1K4eZGmbW0peZvcSRpgJeCSuI6A6PWZYz

https://drive.google.com/open?id=1xIzQrhWrJeid1J8YLFMt8yigFVDA-N8a

https://drive.google.com/open?id=1fZY-tP2MuUYCA7Ur0C0zGTfEBs1XswGQ

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import os
from google.colab import drive

# Mount google drive
DRIVE_MOUNT='/content/gdrive'
drive.mount(DRIVE_MOUNT)


# create folder to write data to
CIS680_FOLDER=os.path.join(DRIVE_MOUNT, 'My Drive', 'CIS680_2019')
HOMEWORK_FOLDER=os.path.join(CIS680_FOLDER, 'HW3a')
os.makedirs(HOMEWORK_FOLDER, exist_ok=True)

# bootstrap environment into place
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

import io
import os
from googleapiclient.http import MediaIoBaseDownload

def download_file(fn, file_id):
    request = drive_service.files().get_media(fileId=file_id)
    downloaded = io.BytesIO()
    downloader = MediaIoBaseDownload(downloaded, request)
    done = False
    while done is False:
        # _ is a placeholder for a progress object that we ignore.
        # (Our file is small, so we skip reporting progress.)
        _, done = downloader.next_chunk()
    
    downloaded.seek(0)

    folder = fn.split('/')
    if len(folder) > 1:
        os.makedirs(folder[0], exist_ok=True)

    with open(fn, 'wb') as f:
        f.write(downloaded.read())

id_to_fn = {
'1uBWazGxSZgWs70JjSWBu-KZwy5sAcxLh': 'hw3_mycocodata_bboxes_comp_zlib.npy',
'18Bh2qwVwdDwu7JK_plrnpAz9KjA5gWkv': 'hw3_mycocodata_img_comp_zlib.h5',
'1K4eZGmbW0peZvcSRpgJeCSuI6A6PWZYz': 'hw3_mycocodata_labels_comp_zlib.npy',
'1xIzQrhWrJeid1J8YLFMt8yigFVDA-N8a': 'hw3_mycocodata_mask_comp_zlib.h5',
'1fZY-tP2MuUYCA7Ur0C0zGTfEBs1XswGQ': 'checkpoint680.pth',
}

# download all files into the vm
for fid, fn in id_to_fn.items():
    print("Downloading %s from %s" % (fn, fid))
    download_file(fn, fid)

ModuleNotFoundError: No module named 'google'

# Load Checkpoint
Here is some sample code to load the pretrained model we are providing

In [4]:
import torchvision
import torch

def pretrained_model_680(checkpoint_file):

    import torchvision
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=False)
    
    model.eval()
    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)
    
    backbone = model.backbone
    rpn = model.rpn
    
    checkpoint = torch.load(checkpoint_file)
    
    backbone.load_state_dict(checkpoint['backbone'])
    rpn.load_state_dict(checkpoint['rpn'])
    
    return backbone, rpn


backbone,rpn = pretrained_model_680('checkpoint680.pth')


E = torch.ones([1,3,800,1088], device='cuda')
backout = backbone(E)
print(len(backout))


from torchvision.models.detection.image_list import ImageList 
im_lis = ImageList(E, [(800,1088)])

rpn.eval()
rpnout = rpn(im_lis,backout)
print(rpnout)

5
([tensor([[  0.0000,   0.0000, 109.5226,  34.7064],
        [  0.0000,   0.0000, 198.6676,  69.8575],
        [  0.0000,   0.0000, 565.8072,  58.1282],
        ...,
        [548.0027, 738.8528, 627.9938, 800.0000],
        [660.0027, 738.8528, 739.9938, 800.0000],
        [804.0027, 738.8528, 883.9938, 800.0000]], device='cuda:0')], {})


# PyTorch Dataset

Copy your HW3a dataset code

# Model Definition

Define the four models

In [0]:
class ROIAlign(torch.nn.Module):
  def __init__(self):
    super().__init__()
    
  def forward(self, X):
    return None

class Classifier(torch.nn.Module):
  def __init__(self):
    super().__init__()
    
  def forward(self, X):
    return None

class Regressor(torch.nn.Module):
  def __init__(self):
    super().__init__()
    
  def forward(self, X):
    return None

class Masks(torch.nn.Module):
  def __init__(self):
    super().__init__()
    
  def forward(self, X):
    return None

# Train your network

It might be good to save checkpoints and reload from the most recent. This is due to time constraints inside of colab.

# Test your network

Did you remember to cut out a test set? If not you really should, test on images your network has never seen.